### Load Packages

In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Chrome webdriver
driver = webdriver.Chrome('/Users/alexcheng/Downloads/chromedriver')

#### Columns

In [11]:
# Column names for the three tables
stat_cols = ['G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%',
             '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 
             'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

shoot_cols = ['G', 'MP', 'FG%', 'AVG_DIST_FGA', '%FGA_2P', '%FGA_0-3ft', 
              '%FGA_3-10ft','%FGA_10-16ft', '%FGA_16ft<3', '%FGA_3P', '2P%',
              '0-3_FG%', '3-10_FG%', '10-16_FG%', '16<3_FG%', '3P%', '%ASTd_2P', 
              '%FGA_DUNK', 'DUNKS', '%ASTd_3P', '%_CORNER3PA', '3P%_CORNER3',
              'HEAVE_ATT', 'HEAVE_MD']

adv_cols = ['G', 'MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%',
            'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
            'WS/48', 'OBPM', 'DPM', 'BPM', 'VORP']

# Join lists together
cols = stat_cols + shoot_cols + adv_cols

#### Load Player List

In [5]:
# Part 1
player_df = pd.read_csv('/Users/alexcheng/Desktop/player_df.csv')
player_list = list(player_df['url'])

# Part 2
player_df_part2 = pd.read_csv('/Users/alexcheng/Desktop/player_df_part2.csv')
player_list_part2 = list(player_df_part2['url'])

# Test list
test_players = pd.read_csv('/Users/alexcheng/Desktop/test_players.csv')
test_list = list(test_players['url'])

#### Temporary

In [6]:
# Temporary list
tmp_list = []

### Web Scraping

In [7]:
for url in test_list:
    # Navigate to URL
    driver.get(url)

    # Check page title has "Basketball" in it
    assert "Basketball" in driver.title

    # Web scraping
    name = driver.find_elements_by_xpath("""//*[@id="meta"]/div[2]/h1""")
    stats = driver.find_elements_by_xpath("""//*[@id="per_game"]/tfoot/tr[1]""")
    advanced = driver.find_elements_by_xpath("""//*[@id="advanced"]/tfoot/tr[1]""")
    

    # Instantiate values and clean values
    for value in name:
        name = value.text
        name = str(name.encode('ascii'))

    for value in stats:
        stats = value.text
        stats = stats[11:].encode('ascii').split()

    for value in shooting:
        shoot_stats = value.text
        shoot_stats = shoot_stats.encode('ascii').split()[5:-2]

    for value in advanced:
        adv_stats = value.text
        adv_stats = adv_stats.encode('ascii').split()[3:]

    # Combine and flatten dataframe
    df = [stats, shoot_stats, adv_stats]

    # Create player stats rows to be joined to dataframe later
    player_stats = [name]
    for sublist in df:
        for val in sublist:
            player_stats.append(val)

    # Append to tmp_list
    tmp_list.append(player_stats)

    # Create dataframe
    df = pd.DataFrame(tmp_list, columns=cols)

### Final Output

In [8]:
df

Player    G   GS    MP   FG  FGA   FG%   3P  3PA   3P%  ...   TOV%  \
0             []   42    0  14.0  1.8  4.5  .396  1.2  3.2  .378  ...   10.2   
1     Quincy Acy  237   51  14.9  1.7  3.5  .495  0.2  0.7  .359  ...   13.0   
2   Steven Adams  282  218  23.2  2.9  5.2  .568  0.0  0.0  .000  ...    9.3   
3  Arron Afflalo  692  518  28.4  4.1  9.2  .451  1.2  3.0  .385  ...   10.4   
4  Alexis Ajinca  275   71  13.2  2.2  4.3  .500  0.0  0.1  .293  ...   15.1   
5   Cole Aldrich  303   23  10.8  1.4  2.6  .531  0.0  0.0   1.4  ...   .141   

   USG%   OWS   DWS    WS WS/48  OBPM   DPM   BPM  VORP  
0  16.4   0.6   0.5   1.1  .088  -0.6  -2.6  -3.2  -0.2  
1  13.8   4.5   2.8   7.3  .099  -1.5   0.0  -1.5   0.5  
2   8.7  18.0  .132  -0.7   1.6   0.9   4.8  None  None  
3  18.0  23.5   9.2  32.8  .080   0.4  -1.3  -1.0   5.2  
4  19.4   2.7   4.2   6.9  .091  -2.8   0.4  -2.5  -0.4  
5  -2.2   3.5   1.3   2.7  None  None  None  None  None  

[6 rows x 66 columns]